In [96]:
import numpy as np
import numpy.matlib 
import math
import matplotlib.pyplot as plt
import scipy.stats

# Inspired by youtube channel D&D Optimised (now d4: D&D Deep Dive)

## Calculate average damage

### Source: D&D Optimised
https://www.youtube.com/watch?v=nk2SIaGXSYo

"The basic formula for calculating average damage is as follows:				
((Average Weapon Damage + Applicable per-hit Damage Modifiers)*(% chance to hit))+((Average Dice Damage on attack)*(% chance to crit))+any once per round damage													
													
Keep in mind that if you have multiple attacks, you need to run the above for each attack - you can simply multiply everything between the outermost parentheses by the # of attacks *if* those attacks are identical,													
but in the case of someone using a Polearm Master Feat, for example, they are not, as the regular attacks would use a D10 for the weapon, wheras the bonus action attack would use a D4 for the weapon."													

### Source: https://www.dndbeyond.com/sources/phb/combat#MakinganAttack
"To make an attack roll, roll a d20 and add the appropriate modifiers. If the total of the roll plus modifiers equals or exceeds the target's Armor Class (AC), the attack hits." 


## Detailed calculations

### Definitions
[roll]: Average Dice Roll
OAC: Opponents Armour Class

### Chance to hit and chance to miss

P(My hit was greater or equal than the Opponents AC) = P([roll]+ hit bonus >= OAC) => P([roll]>= OAC- hit bonus)

P(hit)=P([rol] >= OAC -hit bonus) = [ 20 -(OAC- hit bonus)+1]/20 = 1 -(OAC- hit bonus -1)/20 = 1 - P(miss)

P(miss) = (OAC- hit bonus -1)/20

### Chance to hit with advantage
P (hit on first roll but not on the second)+ P (hit on the second roll but not on the first)+ P(hit on both rolls)=  2  * [ 1 -(OAC- hit bonus -1)/20] * [ (OAC- hit bonus -1)/20]+ [ 1 -(OAC- hit bonus -1)/20] **2 (see also binomial distribution).

The shorter way is to calculate "1 - the chance of missing both rolls". 

P (miss both rolls) = [P(miss)] = [(OAC- hit bonus -1)/20]**2

P( hit with an advantage) = 1 - [(OAC- hit bonus -1)/20]**2

### Chance to hit with advantage and Elven Accuracy (or Lucky)
Elven accuracy, i.e roll one more time: 1- [(OAC- hit bonus -1)/20]**3

### Critical hit  (minimum chance to hit)
Crit hit (natural 20): p(critit)= 1/20

### Critical hit with advantage
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**2

### Minimum chance to hit with advantage and Elven Accuracy (or Lucky)
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**3

### Minimum chance to hit with Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)

In [97]:
1- (1-2/20)

0.09999999999999998

### Minimum chance to hit with advantage, Elven Accuracy (or Lucky) and Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)**3

In [98]:
1- (1-2/20)**3

0.2709999999999999

One way to obtain advantage as a Warlock is to get Devil's Sight and Cast Darkness on oneself (when multiclass with e.g. fighter, one may use the blind fighting style instead of Devil's Sight).

##  Calculating the Probablility of critical hits (rounded to 5 decimal digits).

In [99]:
import dnd_optimisation as dndopt # this is a python file containing the calcuations for critical hits

In [100]:
import importlib  #This is because I am making changes to the file all the time ;) 
importlib.reload(dndopt)  

<module 'dnd_optimisation' from '/Users/eleni/Python Scripts/DnD/dnd_optimisation.py'>

#### Setting advantage, luck_point (related to the lucky feat) and elven_accuracy to false gives the probabity of a natural critical hit (natural 20) 1/20

In [101]:
advantage=False; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [102]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.05

#### Only advantage

In [103]:
advantage=True; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [104]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.0975

#### Advantage and lucky

In [105]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=False

In [106]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.14263

#### Hexblade curse with advantage

In [107]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=True

In [108]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.271

# Comparing different builds

## Warlock (Celestial Patron) with 2 levels of Fighter vs 2 levels of Paladin for maximal damage 

### Assuming the following Ability Scores

In [109]:
AbilityScores= numpy.array([19, 14, 12, 16, 8, 18])

In [110]:
Abilities=['STR', 'DEX', 'CON','INT','WIS', 'CHA']

### Calculate the Ability Score Modifiers

In [111]:
Modifiers = np.floor((AbilityScores - 10)/2)

In [112]:
Modifiers=Modifiers.astype(int)

### What is the Ability Score for Charisma?

In [113]:
AbilityScores[Abilities.index('CHA')]

18

### What is the Modifier for Charisma?

In [114]:
Modifiers[Abilities.index('CHA')]

4

### Define the Character's Level

In [115]:
Level=8;

### Calculate the Proficiency Bonus

In [116]:
ProficiencyBonus=1+np.ceil(Level/4)

In [117]:
ProficiencyBonus=ProficiencyBonus.astype(int)

### Calculate Hit for Spells Attacks casted with Charisma

In [118]:
OtherBonus=1; #Rod of pact keeper

In [119]:
SpellHit=Modifiers[Abilities.index('CHA')]+ProficiencyBonus+OtherBonus

In [120]:
SpellHit

8

### Calculate Hit for Weapon Attacks with Strength

In [121]:
WeaponBonus=1; #Magical +1 weapon

In [122]:
MeleeHit=Modifiers[Abilities.index('STR')]+ProficiencyBonus+WeaponBonus

In [123]:
MeleeHit

8

In [124]:
### Calculate Hit for Weapon Attacks with Dexterity

In [125]:
DexMeleeHit=Modifiers[Abilities.index('DEX')]+ProficiencyBonus+WeaponBonus

In [126]:
DexMeleeHit

6

### The Opponent's Armor Class is:

In [127]:
OAC=15

## Calculate the damage of Eldrich Blast

### Probability to hit an Opponent

#### Probability to hit the Opponent (for 1 dice roll)

In [128]:
PrHit= 1 -(OAC- SpellHit -1)/20 

In [129]:
PrHit

0.7

#### using a function to calculate the probability 
####  probability_of__hitting_opponent(opponent_AC,modifiers,advantage,luck_point,elven_accuracy)

In [130]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,0,0,0)

0.7

#### Probability to miss the Opponent (for 1 dice roll)

In [131]:
PrMiss=(OAC- SpellHit -1)/20 

In [132]:
PrMiss

0.3

#### Probability to hit the opponent with advantage (Ad=2)

In [133]:
Ad=2

In [134]:
PrHitA=1-(PrMiss)**(Ad)

In [135]:
PrHitA

0.91

#### testing function

In [136]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,1,0,0)

0.91

#### Number of Elrdich Blast Rays

In [137]:
Rays=min(math.floor(Level/5)+1,4)

In [138]:
Rays

2

#### In that case the average damage (per ray) will be (with Agonizing Blast) 1d10+ Charisma Modifier

In [139]:
AverageAgonisingBlastDamage=5.5

In [140]:
AverageDamagePerRay=AverageAgonisingBlastDamage+Modifiers[Abilities.index('CHA')]

In [141]:
AverageDamagePerRay

9.5

Additional modifier for Hex spell

In [142]:
HexAverageDamage=3.5

### Average damage, taking into account critical hits, parametrised by the number of rays

For two rays only the formula would be 

 2 x Probability that only one ray will hit x damage + Probability that both rays will hit x 2 x damage + 2 x Probability that only one dice will crit hit x damage  + Probability that both dices will hit x 2 x damage 

In [143]:
Critit=1/20;

In [144]:
2 * PrHit * PrMiss * AverageDamagePerRay+ 2* (Critit) *(1-Critit) * AverageDamagePerRay+ PrHit**2 * 2* AverageDamagePerRay + 2* (Critit) *(Critit) * AverageDamagePerRay

14.249999999999998

In order to easily generalise to higher number of rays we use the binomial distribution. we check for 2 Rays to see the same result as above.

Without taking into account Crit Hit

In [145]:
ADNCR = scipy.stats.binom.pmf(1,2,PrHit)*AverageDamagePerRay + scipy.stats.binom.pmf(2,2,PrHit)* 2* AverageDamagePerRay 

In [146]:
ADNCR

13.3

Calculating Crit Hit

In [147]:
ACHD= scipy.stats.binom.pmf(1,2,Critit)*AverageDamagePerRay + scipy.stats.binom.pmf(2,2,Critit)* 2* AverageDamagePerRay 

In [148]:
ACHD

0.9500000000000003

Adding the two together

In [149]:
TAD=ADNCR+ACHD

In [150]:
TAD

14.250000000000002

#### Calculating for arbitrary number of Rays

In [151]:
TotalDamage=0;

In [152]:
indexRay = 1
while indexRay < (Rays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,Rays,PrHit)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,Rays,Critit)*indexRay*AverageDamagePerRay
    indexRay += 1

In [153]:
TotalDamage

14.25

### Average damage with  Advantage

In [154]:
n=1

In [155]:
TotalRays=n*Rays

In [156]:
CrititA=1-(1-1/20)**2

In [157]:
TotalDamage=0

In [158]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*AverageDamagePerRay
    indexRay += 1

In [159]:
TotalDamage

19.142500000000005

### Average damage with Advantage and Hex

In [160]:
n=1

In [161]:
TotalRays=n*Rays

In [162]:
TotalDamage=0

In [163]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*(AverageDamagePerRay+HexAverageDamage)+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*(AverageDamagePerRay+HexAverageDamage)
    indexRay += 1

In [164]:
TotalDamage

26.195000000000004

### Average damage with  n repetitions (e.g. Action Surge n=2)

In [165]:
n=2;

In [166]:
TotalRays=n*Rays

In [167]:
TotalDamage=0

In [168]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHit)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,Critit)*indexRay*AverageDamagePerRay
    indexRay += 1

In [169]:
TotalDamage

28.500000000000007

### Average damage with  n repetitions  (e.g. Action Surge n=2) and Advantage

For rolls with advantage I need to use in the formula the probability of hitting the target with advantage

In [170]:
n=2;

In [171]:
TotalRays=n*Rays

In [172]:
TotalDamage=0

In [173]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*AverageDamagePerRay
    indexRay += 1

In [174]:
  TotalDamage

38.285000000000004

### Average damage with  n repetitions  (e.g. Action Surge n=2), Advantage and Hex

In [226]:
n=2;

In [227]:
TotalRays=n*Rays

In [228]:
TotalRays

4

In [229]:
TotalDamage=0

In [230]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*(AverageDamagePerRay+HexAverageDamage)+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*(AverageDamagePerRay+HexAverageDamage)
    indexRay += 1

In [231]:
TotalDamage

52.39

#### Testing the function average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

In [232]:
repetitions=4; oddbonus=0;average_damage_per_hit=AverageDamagePerRay+HexAverageDamage;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [233]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,oddbonus,advantage,luck_point,elven_accuracy,hexblade_curse)

52.39

### Without Action surge, only advantage with two eldrich blast rays (repetitions=2)

In [234]:
repetitions=Rays;  oddbonus=0; average_damage_per_hit=AverageDamagePerRay+HexAverageDamage;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [235]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,oddbonus,advantage,luck_point,elven_accuracy,hexblade_curse)

26.195

## Paladin built: calculate the damage of Melee (+1 weapon) + Hex + Green Flame Blade+ two Smites with Advantage

#### Magical +1 Long sword damage 1d8+1 (+4) + Green flame blade 

In [238]:
WeaponAverageDamage=4.5+1+Modifiers[Abilities.index('STR')]+2*4.5+4

In [239]:
WeaponAverageDamage

22.5

In [240]:
PrHit= 1 -(OAC- MeleeHit -1)/20 

In [241]:
PrHit

0.7

In [242]:
PrMiss=1-PrHit

In [243]:
Ad=2

In [244]:
PrHitAd=1-(PrMiss)**(Ad)

In [245]:
PrHitAd

0.9099999999999999

#### Divine smite (3rd level slot) 4d8 average 4.5*4

In [246]:
Smite=4.5*4

#### bonus smite: I am assuming  searing smite and a celestial warlock's bonus to radient damage

In [247]:
BonusSmite=3*3.5+Modifiers[Abilities.index('CHA')]

In [248]:
HexAverageDamage

3.5

In [249]:
TotalDamage= (WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage)*PrHitAd+ (WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage) *CrititA

In [250]:
TotalDamage

58.93874999999999

#### Using the function average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

In [251]:
repetitions=1; oddbonus=0;average_damage_per_hit=WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage;opponent_AC=OAC;modifiers=MeleeHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [252]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,oddbonus,advantage,luck_point,elven_accuracy,hexblade_curse)

58.93875

## Higher than the fighter but asks for the consumption of two spell slots of the highest level and that the characters casts first searing smite as bonus action. Slightly more complecated mechanics. Action surge will not consume spell slots.